In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path='/content/gdrive/MyDrive/Datasets/'
movies_df=pd.read_csv(path+'movies.csv',dtype={'movieId': 'int32', 'title': 'str'})
ratings_df= pd.read_csv(path+'ratings.csv',usecols=['userId', 'movieId', 'rating'],
            dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [5]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings_df.head()


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
df = pd.merge(ratings_df,movies_df,on='movieId')
df

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,Blair Witch (2016),Horror|Thriller


In [8]:
df['rating'] = df['rating'].fillna(0)

In [10]:
movie_ratingCount= (df.
     groupby('title')['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'}))
movie_ratingCount.head(10)

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
5,'Tis the Season for Love (2015),1
6,"'burbs, The (1989)",17
7,'night Mother (1986),1
8,(500) Days of Summer (2009),42
9,*batteries not included (1987),7


In [11]:
movie_with_totalRatingCount= df.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
movie_with_totalRatingCount.head()

,userId,movieId,rating,title,genres,totalRatingCount
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [12]:
popularity_threshold = 50
query_string = f'totalRatingCount >= {popularity_threshold}'
rating_popular_movie = movie_with_totalRatingCount.query(query_string)
rating_popular_movie

,userId,movieId,rating,title,genres,totalRatingCount
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
...,...,...,...,...,...,...
79246,603,1997,4.0,"Exorcist, The (1973)",Horror|Mystery,53
79247,606,1997,3.0,"Exorcist, The (1973)",Horror|Mystery,53
79248,607,1997,5.0,"Exorcist, The (1973)",Horror|Mystery,53
79249,608,1997,4.5,"Exorcist, The (1973)",Horror|Mystery,53


In [13]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156234 sha256=1cf680373a8bd9b731c86a1ad60001189b516e87f8f13a9e0d95c5256333d643
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [14]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np

# Load your dataset and prepare it as you did in your code

def train_item_based_collaborative_filtering_model(dataset):
    # Step 1: Define the Reader and load the dataset
    reader = Reader(rating_scale=(0, 5))
    data = Dataset.load_from_df(dataset[['userId', 'movieId', 'rating']], reader)

    # Step 2: Split the dataset into a train set and a test set
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

    # Step 3: Build the user-item interaction matrix
    trainset = data.build_full_trainset()

    # Step 4: Perform SVD on the user-item interaction matrix
    model = SVD(n_factors=100, random_state=42)  # You can adjust the number of factors
    model.fit(trainset)

    # Step 5: Get the item-factor matrix (V)
    V = model.qi

     # Step 6: Compute item-item similarity matrix using cosine similarity
    item_sim_matrix = np.dot(V, V.T)

     # Step 6: Compute item-item similarity matrix using cosine similarity
    item_sim_matrix = np.dot(V, V.T)

    # Step 7: Make predictions on the test set
    predictions = model.test(testset)

    # Step 8: Measure RMSE and MAE
    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)

    return model


In [16]:
def get_item_based_recommendations(movie_name, item_sim_matrix, item_id_mapping, num_recommendations=10):

     movie_id = movies_df[movies_df['title'] == movie_name]['movieId'].values[0]
     # Get the row index in the V matrix corresponding to the given movie ID
     if movie_id in item_id_mapping:
        movie_index = item_id_mapping[movie_id]
     else:
        raise ValueError(f"Movie ID {movie_id} not found in the mapping.")

     # Get the similarity scores for the given movie
     movie_similarity_scores = item_sim_matrix[movie_index]

     # Sort movies by similarity in descending order
     similar_movie_indices = np.argsort(movie_similarity_scores)[::-1]

    # Exclude the input movie from recommendations
     similar_movie_indices = similar_movie_indices[similar_movie_indices != movie_index]

     # Get the top N most similar movies
     top_similar_movies = similar_movie_indices[:num_recommendations]

     # Convert movie indices back to movie IDs using the mapping
     recommended_movie_ids = [item_id for item_id, index in item_id_mapping.items() if index in top_similar_movies]

     # Get the movie titles for recommendations
     recommended_movie_titles = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]['title'].tolist()

     return recommended_movie_titles


model = train_item_based_collaborative_filtering_model(df)
item_sim_matrix = np.dot(model.qi, model.qi.T)
# Create a mapping between movie IDs and their row indices in the V matrix
movie_id_mapping = {movie_id: idx for idx, movie_id in enumerate(movies_df['movieId'])}

movie_name= 'X-Men: The Last Stand (2006)'
recommendations = get_item_based_recommendations(movie_name, item_sim_matrix, movie_id_mapping)
for idx, movie in enumerate(recommendations, start=1):
        print(f"{idx}. {movie}")

RMSE: 0.6410
MAE:  0.4968
1. Nell (1994)
2. Victor/Victoria (1982)
3. Lord of the Rings, The (1978)
4. Dirty Rotten Scoundrels (1988)
5. Everybody's Famous! (Iedereen beroemd!) (2000)
6. Dolce Vita, La (1960)
7. Go for Zucker! (Alles auf Zucker!) (2004)
8. Dark Knight, The (2008)
9. Magic Mike (2012)
10. Boundaries (2018)
